In [1]:
%pwd

'd:\\2025_Projects\\Medical_chatbot\\AI-Medical-Chatbot-Assistance\\research'

In [2]:
import os
os.chdir("../")

In [3]:
%pwd

'd:\\2025_Projects\\Medical_chatbot\\AI-Medical-Chatbot-Assistance'

In [4]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [5]:
def load_pdf_file(data):
    loader=DirectoryLoader(data,
                           glob="*.pdf",
                           loader_cls=PyPDFLoader)
    documents=loader.load()

    return documents
    

In [6]:
extracted_data=load_pdf_file(data='Data/')

In [9]:
#extracted_data

In [13]:
#Split the data into text chunks
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [14]:
text_chunks=text_split(extracted_data)
print("Length of Text Chunks", len(text_chunks))

Length of Text Chunks 5859


In [15]:
from langchain.embeddings import HuggingFaceEmbeddings


In [16]:
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [17]:
embeddings = download_hugging_face_embeddings()

C:\Users\HP\AppData\Local\Temp\ipykernel_10024\2604741814.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
d:\Anaconda\envs\medichatbot\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [18]:
query_result = embeddings.embed_query("Hello world")
print("Length: ", len(query_result))

Length:  384


In [19]:
query_result

[-0.034477200359106064,
 0.031023219227790833,
 0.0067349993623793125,
 0.02610897459089756,
 -0.03936201333999634,
 -0.16030248999595642,
 0.06692397594451904,
 -0.006441470701247454,
 -0.04745051637291908,
 0.014758859761059284,
 0.07087533175945282,
 0.05552757531404495,
 0.019193289801478386,
 -0.026251329109072685,
 -0.010109508410096169,
 -0.026940548792481422,
 0.022307483479380608,
 -0.022226618602871895,
 -0.14969265460968018,
 -0.01749304309487343,
 0.007676269859075546,
 0.05435226485133171,
 0.003254482988268137,
 0.03172600269317627,
 -0.0846213772892952,
 -0.02940598875284195,
 0.051595672965049744,
 0.04812409356236458,
 -0.003314794274047017,
 -0.05827922374010086,
 0.041969284415245056,
 0.022210709750652313,
 0.1281888633966446,
 -0.022338956594467163,
 -0.011656241491436958,
 0.06292833387851715,
 -0.032876305282115936,
 -0.09122610837221146,
 -0.03117538057267666,
 0.05269957706332207,
 0.047034792602062225,
 -0.08420304954051971,
 -0.0300561785697937,
 -0.020744718

In [20]:
from dotenv import load_dotenv
load_dotenv()

True

In [21]:
import os

PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY')
OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY')

In [22]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "medichatbot1"

pc.create_index(
    name=index_name,
    dimension=384,
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
)

PineconeApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2025-04', 'x-cloud-trace-context': 'ef48f4335d6c001d46833b4e037aa30c', 'date': 'Wed, 09 Jul 2025 09:07:59 GMT', 'server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}


In [23]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [24]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings,
)

In [25]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)


In [26]:
docsearch

In [27]:
retriever = docsearch.as_retriever(search_type ="similarity", search_kwargs={"k":3})

In [28]:
retrieved_docs = retriever.invoke("What is Acne?")

In [29]:
retrieved_docs

[Document(id='7d55d045-636b-4cd0-8bdd-ec00dd6916ad', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 39.0, 'page_label': '40', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'Data\\Medical_book.pdf', 'total_pages': 637.0}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'),
 Document(id='593785c4-16f1-45a4-904d-a34550e0be17', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 39.0, 'page_label': '40', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'Data\\Medical_book.pdf', 'total_pages': 637.0}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'),
 Document(id='b970a229-5553-4352-bf8d-2c52757a37ce', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 38.0, 

In [30]:
from langchain_openai import OpenAI
llm = OpenAI(temperature=0.4, max_tokens=500)

In [32]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate


system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise. "
    "\n\n"
    "{context}"
)

# Corrected: Pass a LIST of tuples to from_messages
prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{input}"),
])



In [33]:
quection_answer_chain = create_stuff_documents_chain(llm,prompt)
rag_chain = create_retrieval_chain(retriever, quection_answer_chain)

In [34]:
response = rag_chain.invoke({"input": "What is Acne?"})
print(response["answer"])



Acne is a skin disorder that involves inflammation of the sebaceous glands. It is also known as acne vulgaris. It is a common condition that can affect both men and women.
